In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 224 

In [4]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 224  # DenseNet121 prefers 224x224

class PneumoniaDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = []
        self.targets = []
        self.transform = transform

        for label in labels:
            path = os.path.join(data_dir, label)
            class_idx = labels.index(label)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                if not img_name.lower().endswith((".jpg",".jpeg",".png")):
                    continue
                self.data.append(img_path)
                self.targets.append(class_idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        image = cv2.imread(img_path)
        if image is None:
            image = np.zeros((img_size, img_size, 3), dtype=np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (img_size, img_size))

        if self.transform:
            image = self.transform(image)

        label = self.targets[idx]
        return image, label


In [5]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(img_size, scale=(0.8,1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_dataset = PneumoniaDataset(r"D:\datasets\chest_xray\chest_xray\train", transform=train_transform)
val_dataset   = PneumoniaDataset(r"D:\datasets\chest_xray\chest_xray\val", transform=val_transform)
test_dataset  = PneumoniaDataset(r"D:\datasets\chest_xray\chest_xray\test", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
# Freeze base layers
for param in model.features.parameters():
    param.requires_grad = False

# Modify classifier
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 1),
    nn.Sigmoid()
)
model = model.to(device)

In [7]:
criterion = nn.BCELoss()

# Handle class imbalance
all_labels = np.array(train_dataset.targets)
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(all_labels), y=all_labels)
weights = torch.tensor(class_weights, dtype=torch.float).to(device)
# Optional: can use WeightedRandomSampler for better balancing
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [9]:
# -------------------
epochs = 10
best_val_acc = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for imgs, labels_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        imgs = imgs.to(device)
        labels_batch = labels_batch.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        preds = (outputs > 0.5).float()
        correct += (preds == labels_batch).sum().item()
        total += labels_batch.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for imgs, labels_batch in val_loader:
            imgs = imgs.to(device)
            labels_batch = labels_batch.float().unsqueeze(1).to(device)
            outputs = model(imgs)
            preds = (outputs > 0.5).float()
            val_correct += (preds == labels_batch).sum().item()
            val_total += labels_batch.size(0)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        torch.save(model.state_dict(), "pneumonia_densenet121.pt")
        best_val_acc = val_acc

# -------------------

Epoch 1/10: 100%|██████████| 163/163 [01:11<00:00,  2.28it/s]


Epoch 1: Train Loss=0.2197, Train Acc=0.9166, Val Acc=0.8125


Epoch 2/10: 100%|██████████| 163/163 [01:14<00:00,  2.18it/s]


Epoch 2: Train Loss=0.2052, Train Acc=0.9247, Val Acc=0.8125


Epoch 3/10: 100%|██████████| 163/163 [01:17<00:00,  2.10it/s]


...

Epoch 8: Train Loss=0.1534, Train Acc=0.9423, Val Acc=0.8750


Epoch 9/10: 100%|██████████| 163/163 [01:15<00:00,  2.15it/s]


Epoch 9: Train Loss=0.1610, Train Acc=0.9333, Val Acc=1.0000


Epoch 10/10: 100%|██████████| 163/163 [01:20<00:00,  2.02it/s]


Epoch 10: Train Loss=0.1524, Train Acc=0.9404, Val Acc=0.9375


In [10]:
model.load_state_dict(torch.load("pneumonia_densenet121.pt"))
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for imgs, labels_batch in test_loader:
        imgs = imgs.to(device)
        labels_batch = labels_batch.to(device)
        outputs = model(imgs)
        preds = (outputs > 0.5).int()
        y_true.extend(labels_batch.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=labels))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

C:\Users\Ekaansh\AppData\Local\Temp\ipykernel_27420\2606734791.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("pneumonia_densenet121.pt


Classification Report:
               precision    recall  f1-score   support

   PNEUMONIA       0.86      0.95      0.90       390
      NORMAL       0.90      0.75      0.82       234

    accuracy                           0.88       624
   macro avg       0.88      0.85      0.86       624
weighted avg       0.88      0.88      0.87       624


Confusion Matrix:
 [[370  20]
 [ 58 176]]
